In [1]:
import json

# Load the dataset
with open('waste_disposal_dataset.json') as f:
    data = json.load(f)

# Extract all unique categories
unique_categories = sorted(set(item["category"] for item in data))

# Print them
print("Unique categories:")
for category in unique_categories:
    print("-", category)


Unique categories:
- Chemicals
- E-waste
- Food packaging
- Garden waste
- Hard waste
- Hazardous waste
- Household items
- Household waste
- Medical waste
- Organic waste
- Packaging
- Paper
- Soft plastics


In [2]:
x = 0
for i in range(len(data)):
  x = (data[i]['item'])
  print(x)

print(len(data))

Aerosol cans
Adhesive strips (band aids)
Aluminium cans
Aluminium foil
Aluminium trays
Anti-freeze
Asbestos
Australia Post satchels
Baby items
Balloons
Baskets
Batteries (car)
Batteries (household)
Bicycles
Biscuit trays
Blankets and bedding
Blister packs (medicinal)
Bottles (glass, plastic, metal)
Branches
Bread bags
Bubble wrap
Cables (computer, charging, electronic)
Cameras
Cans (steel, aluminium)
Car parts and bodies
Cardboard
Cartons (milk and juice)
Carpet
Cassette tapes
CDs
Ceramics (vases, pots, etc)
Cellophane
Chargers
Chemicals
Chip packets
Chlorine
Christmas trees
Cigarette butts
Cling wrap
Clothing and shoes
Coffee cups and lids (single use)
Coffee pods / capsules
Coffee machines
Computers and accessories
Cooking oil
Crockery
Cutlery (metal)
Cutlery (plastic)
Cupboards
Detergent bottles (plastic)
Deodorant cans, roll-ons, sprays
Diabetic testing strip
Dishwasher
Disposable coffee cups
Dog faeces
Door mat
Dryer
DVDs
DVD player
E-waste
Fabric
Fire extinguishers
Fluorescent tu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [3]:
!pip install openai
!pip install openai pandas


In [4]:
# 📦 Import Libraries
import json
import urllib.parse  # ✅ Add this line
from openai import OpenAI


In [ ]:


import openai
import pandas as pd
import json

# Load your API key here
from openai import OpenAI
client = OpenAI(api_key="")  # Replace with your API key

# Load dataset
with open('/content/waste_disposal_dataset.json') as f:
    waste_data = json.load(f)

# Additional info per category
additional_notes = {
    "Hard waste": """
🏠 **City of Melbourne Hard Waste Collection**

Residents are entitled to **one free hard waste collection per financial year** (maximum volume: 1 cubic metre – 1m x 1m x 1m). If you live in a high-rise, contact your building manager to organize the booking.

📦 **Accepted Items Include**:
- Scrap iron, steel, and metal items (e.g., old barbecues)
- Pottery and chinaware
- Securely wrapped sheet glass clearly marked as 'glass'
- Mattresses, small rugs (max 1.5m length), tied and rolled
- All sizes of e-waste (TVs, computers, etc.)
- Whitegoods and appliances (fridges, stoves, dishwashers – **doors must be removed**)
- Household furniture, tools, sporting goods, toys, bicycles
- Timber < 1m in length (max 10 pieces)
- Hot water units

📅 **Collection Rules**:
- Bookings are essential and can be made up to 2 months in advance
- Place items at the kerb **the day before** your collection
- Collection trucks **will not enter private property**

🔗 Book here: https://cityofmelbourne-hardwaste.cleanaway.com.au/
🔁 Items not accepted? Consider donating, or use Dynon Road Recycling Centre (fees may apply)
""",

    "Hazardous waste": """
☣️ **Hazardous & Chemical Waste Disposal**

Do **not** place any chemicals in your landfill or recycling bins.

✅ **Safe Disposal Option**:
Drop off household chemicals for free at your closest **Detox Your Home** collection site, organized by **Sustainability Victoria**.

🧪 **Accepted Items**:
- Paint, solvents, motor oil, cleaning products, garden chemicals, etc.

⚠️ **Safety Guidelines**:
- Max size per container: **20kg or 20L**
- **Do not decant**; bring chemicals in **sealed disposable containers**
- Some chemicals may be restricted (check their website before attending)

🚛 **Mobile collection service** is also available with broader acceptance range.

🔗 More info & locations: https://www.sustainability.vic.gov.au/detoxyourhome
""",

    "E-waste": """
🔌 **Electronic Waste (E-Waste) Disposal**

E-waste must **not be placed in household bins**. It includes anything with a plug, battery, or cord.

📏 **Categories**:
- **Small E-waste**: Mobile phones, power banks, charging cables, household batteries
  → Drop-off at:
    - North Melbourne Library (66 Errol St)
    - Boyd Community Hub (207 City Rd)
    - East Melbourne Library
    - Library at The Dock
    - Kathleen Syme Library
    - Kensington Town Hall
    - Dynon Road Waste & Recycling Centre

- **Medium E-waste**: Monitors, small TVs, vacuums, kettles, microwaves
  → Drop-off at:
    - Dynon Road Recycling Centre
    - Kathleen Syme Library
    - Library at The Dock

- **Large E-waste**: TVs, washing machines, computers (large)
  → **Book free hard waste collection**:
    https://hardwastecom.citywide.com.au/

🛑 **Never dispose of e-waste in landfill or recycling bins.**
"""
}



In [ ]:
# Function to call LLM
def call_llm(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant for the City of Melbourne's waste and recycling guide. "
                    "Based on the item information below, provide a friendly and informative response to a user "
                    "asking how to dispose of this item. Include details like drop-off instructions, recycling advice, "
                    "and any important conditions. If additional notes are provided, summarize them clearly. "
                    "Avoid repeating labels like 'Item:' or 'Category:' in your answer. "
                    "If a Google Maps link is provided, show it clearly as a separate line in the response."
                )
            },
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# Main recycling info handler
def get_recycling_info(user_query):
    user_query = user_query.lower()

    for item in waste_data:
        keywords = [item["item"].lower()] + [alias.lower() for alias in item.get("aliases", [])]
        if any(kw in user_query for kw in keywords):
            base_info = f"""
Item: {item['item']}
Category: {item['category']}
Bin Type: {item['bin_type'] or 'Not specified'}
Recyclable: {'Yes' if item['recyclable'] else 'No'}
Instructions: {item['instructions']}
"""

            map_links = []

            if item.get("drop_off_required") and item.get("drop_off_locations"):
                locations = []
                for loc in item["drop_off_locations"]:
                    name = loc["name"]
                    address = loc["address"]
                    suburb = loc["suburb"]
                    if "see website" not in address.lower() and "online" not in address.lower():
                        query = f"{address} {suburb}".strip().replace(" ", "+")
                        maps_link = f"https://www.google.com/maps/search/?api=1&query={query}"
                        location_str = f"- {name}, {address}" + (f" ({suburb})" if suburb else "") + f"\n  Google Maps: {maps_link}"
                        map_links.append(f"{name}: {maps_link}")
                    else:
                        location_str = f"- {name}, {address}" + (f" ({suburb})" if suburb else "")
                    locations.append(location_str)
                base_info += f"\nDrop-off Locations:\n" + "\n".join(locations) + "\n"

            if item.get("extra_notes"):
                base_info += f"\nExtra Notes:\n{item['extra_notes'].strip()}\n"

            if item["category"] in additional_notes:
                base_info += f"\nCategory Info:\n{additional_notes[item['category']].strip()}"

            prompt = f"""A user asked how to dispose of: {item['item']}.

Here is the relevant information:
{base_info.strip()}

Please explain this in a clear and helpful way."""

            response = call_llm(prompt)

            # Print LLM response
            print(response)

            # Print Google Maps links clearly
            if map_links:
                print("\n📍 Google Maps Links:")
                for link in map_links:
                    print(link)

            return  # Don't return LLM response — already printed

    print("Sorry, I couldn't find recycling information for that item.")


In [ ]:
query = "carpets"  # Try: "TV units", "Tupperware", "Chemicals", etc.
get_recycling_info(query)

To dispose of carpet, you can take it to the Dynon Road Waste and Recycling Centre located at 437 Dynon Road in West Melbourne. Charges may apply for drop-off, so it's good to keep that in mind.

City of Melbourne offers residents one free hard waste collection per financial year. When disposing of items like carpet at the drop-off location, make sure you comply with the rules such as securely wrapping the carpet and taking it to the designated facility.

If you need to schedule a hard waste collection, you can book it online up to two months in advance and place the items at the kerb the day before collection. Remember that trucks will not enter private property for collection.

For more details and to schedule a collection, you can visit: https://cityofmelbourne-hardwaste.cleanaway.com.au/ 

If you have other items that are not accepted, consider donating them or utilizing the Dynon Road Recycling Centre for disposal (fees may apply). 

Drop-off Location:
- Dynon Road Waste and Recyc

After mounting your drive, you can access your files at `/content/drive/MyDrive/`.

In [5]:
!pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 63.0 MB/s eta 0:00:00


In [ ]:
from rapidfuzz import fuzz

def get_recycling_info(user_query, threshold=70):
    user_query = user_query.lower()
    found_items = []

    for item in waste_data:
        keywords = [item["item"].lower()] + [alias.lower() for alias in item.get("aliases", [])]

        for kw in keywords:
            score = fuzz.partial_ratio(user_query, kw)  # partial match works better for long queries
            if score >= threshold:
                found_items.append((item, score))
                break  # stop after first good alias match for this item

    if not found_items:
        print("Sorry, I couldn't find recycling information for that item.")
        return

    for item, score in found_items:
        base_info = f"""
Item: {item['item']}
Category: {item['category']}
Bin Type: {item['bin_type'] or 'Not specified'}
Recyclable: {'Yes' if item['recyclable'] else 'No'}
Instructions: {item['instructions']}
"""

        if item.get("extra_notes"):
            base_info += f"\nExtra Notes:\n{item['extra_notes'].strip()}\n"

        if item["category"] in additional_notes:
            base_info += f"\nCategory Info:\n{additional_notes[item['category']].strip()}"

        # LLM response
        prompt = f"""A user asked: "{user_query}".

Here is the relevant information:
{base_info.strip()}

Please explain this in a clear and helpful way."""

        response = call_llm(prompt)

        print("🟢 Match found:", item['item'], f"(confidence {score}%)")
        print(response)
        print("\n" + "="*50 + "\n")


In [ ]:
query = "where to dispose of carpets"
get_recycling_info(query)


🟢 Match found: Carpet (confidence 100.0%)
You can dispose of carpets as hard waste by utilizing the City of Melbourne's hard waste collection service. Simply book a collection using the link provided, ensuring that the carpet is neatly bundled and within the maximum volume limit of 1 cubic meter. Place the carpet at the kerb the day before your scheduled collection. Note that charges may apply for drop-off.

📍 Drop-off: City of Melbourne Hard Waste Collection  
🔗 Book here: [City of Melbourne Hard Waste Collection Booking](https://cityofmelbourne-hardwaste.cleanaway.com.au/)


🟢 Match found: Coffee cups and lids (single use) (confidence 70.0%)
You can dispose of coffee cups and lids in the following way:

1. Remove the lid and place it in the recycling (yellow lid) bin.
2. Place the cup in the general waste (red lid) bin or drop it off at 7/11 stores for recycling.

7/11 partners with Simply Cups for recycling coffee cups. You can find the nearest 7/11 store by using the following Goog

In [6]:

# Install deps (Colab)
!pip -q install rapidfuzz gradio pandas openai

import re, json, math, urllib.parse
from collections import defaultdict
from rapidfuzz import process, fuzz
import pandas as pd

# --- Load your dataset ---
with open('/content/waste_disposal_dataset.json') as f:
    waste_data = json.load(f)

# --- Optional: your additional notes dict (reuse yours as-is) ---
additional_notes = {
    "Hard waste": """
🏠 **City of Melbourne Hard Waste Collection**

Residents are entitled to **one free hard waste collection per financial year** (maximum volume: 1 cubic metre – 1m x 1m x 1m). If you live in a high-rise, contact your building manager to organize the booking.

📦 **Accepted Items Include**:
- Scrap iron, steel, and metal items (e.g., old barbecues)
- Pottery and chinaware
- Securely wrapped sheet glass clearly marked as 'glass'
- Mattresses, small rugs (max 1.5m length), tied and rolled
- All sizes of e-waste (TVs, computers, etc.)
- Whitegoods and appliances (fridges, stoves, dishwashers – **doors must be removed**)
- Household furniture, tools, sporting goods, toys, bicycles
- Timber < 1m in length (max 10 pieces)
- Hot water units

📅 **Collection Rules**:
- Bookings are essential and can be made up to 2 months in advance
- Place items at the kerb **the day before** your collection
- Collection trucks **will not enter private property**

🔗 Book here: https://cityofmelbourne-hardwaste.cleanaway.com.au/
🔁 Items not accepted? Consider donating, or use Dynon Road Recycling Centre (fees may apply)
""",

    "Hazardous waste": """
☣️ **Hazardous & Chemical Waste Disposal**

Do **not** place any chemicals in your landfill or recycling bins.

✅ **Safe Disposal Option**:
Drop off household chemicals for free at your closest **Detox Your Home** collection site, organized by **Sustainability Victoria**.

🧪 **Accepted Items**:
- Paint, solvents, motor oil, cleaning products, garden chemicals, etc.

⚠️ **Safety Guidelines**:
- Max size per container: **20kg or 20L**
- **Do not decant**; bring chemicals in **sealed disposable containers**
- Some chemicals may be restricted (check their website before attending)

🚛 **Mobile collection service** is also available with broader acceptance range.

🔗 More info & locations: https://www.sustainability.vic.gov.au/detoxyourhome
""",

    "E-waste": """
🔌 **Electronic Waste (E-Waste) Disposal**

E-waste must **not be placed in household bins**. It includes anything with a plug, battery, or cord.

📏 **Categories**:
- **Small E-waste**: Mobile phones, power banks, charging cables, household batteries
  → Drop-off at:
    - North Melbourne Library (66 Errol St)
    - Boyd Community Hub (207 City Rd)
    - East Melbourne Library
    - Library at The Dock
    - Kathleen Syme Library
    - Kensington Town Hall
    - Dynon Road Waste & Recycling Centre

- **Medium E-waste**: Monitors, small TVs, vacuums, kettles, microwaves
  → Drop-off at:
    - Dynon Road Recycling Centre
    - Kathleen Syme Library
    - Library at The Dock

- **Large E-waste**: TVs, washing machines, computers (large)
  → **Book free hard waste collection**:
    https://hardwastecom.citywide.com.au/

🛑 **Never dispose of e-waste in landfill or recycling bins.**
"""
}



# ---------- Text utils ----------
_word_re = re.compile(r"[a-z0-9]+")

def normalize(s: str) -> str:
    s = s.lower()
    return " ".join(_word_re.findall(s))

def maybe_singular(pl: str) -> str:
    # very light heuristic
    return pl[:-1] if pl.endswith('s') and len(pl) > 3 else pl

def split_multi_query(q: str):
    # split on commas or " and "
    parts = re.split(r",| and ", q, flags=re.I)
    return [p.strip() for p in parts if p.strip()]

# ---------- Build searchable index ----------
# We give weights to different fields so item name/alias outranks category hits.
INDEX = []     # list of (index_key, item_idx, field, weight)
INDEX_KEYS = []  # just the strings for vectorized matching

FIELD_WEIGHTS = {
    "item": 1.0,
    "alias": 0.95,
    "category": 0.65,
}

for idx, it in enumerate(waste_data):
    # item name
    key = normalize(it["item"])
    INDEX.append((key, idx, "item", FIELD_WEIGHTS["item"]))
    INDEX_KEYS.append(key)

    # aliases
    for al in it.get("aliases", []):
        k = normalize(al)
        INDEX.append((k, idx, "alias", FIELD_WEIGHTS["alias"]))
        INDEX_KEYS.append(k)

    # category (helps queries like "chemicals", "e-waste")
    if it.get("category"):
        k = normalize(it["category"])
        INDEX.append((k, idx, "category", FIELD_WEIGHTS["category"]))
        INDEX_KEYS.append(k)

# For speed, keep a parallel list of metadata for the INDEX rows:
INDEX_META = [{"item_idx": idx, "field": fld, "weight": w} for (_, idx, fld, w) in INDEX]

# ---------- Core search ----------
def score_one_subquery(subq_norm: str, score_cutoff=60, top_k=50):
    """
    Returns a dict: item_idx -> {"score": aggregated_score, "hits": [(index_row, score, method), ...]}
    We ensemble two scorers and keep the max per index row, then aggregate to item.
    """
    # Vectorized matching over INDEX_KEYS
    # 1) token_set_ratio (handles word order & duplicates well)
    hits_a = process.extract(
        subq_norm, INDEX_KEYS,
        scorer=fuzz.token_set_ratio,
        score_cutoff=score_cutoff,
        limit=top_k
    )
    # 2) partial_ratio (good for substrings)
    hits_b = process.extract(
        subq_norm, INDEX_KEYS,
        scorer=fuzz.partial_ratio,
        score_cutoff=score_cutoff,
        limit=top_k
    )

    # Merge hits: keep max score per INDEX row
    merged = {}
    for (idx_key, score, pos) in hits_a:
        merged[pos] = ("token_set_ratio", score)
    for (idx_key, score, pos) in hits_b:
        prev = merged.get(pos)
        if (not prev) or (score > prev[1]):
            merged[pos] = ("partial_ratio", score)

    # Aggregate to items with weights + small bonuses
    by_item = defaultdict(lambda: {"score": 0.0, "hits": []})

    for pos, (method, base_score) in merged.items():
        meta = INDEX_META[pos]
        row_text = INDEX_KEYS[pos]
        weight = meta["weight"]
        item_idx = meta["item_idx"]

        # Bonuses for high-precision signals
        bonus = 0.0
        if row_text in subq_norm:
            bonus += 5.0  # substring present
        if subq_norm.startswith(row_text[: max(3, len(row_text)//2)]):
            bonus += 2.0  # prefix-ish

        # Weighted score
        wscore = (base_score * weight) + bonus

        # Keep the best contribution from this index row
        by_item[item_idx]["score"] = max(by_item[item_idx]["score"], wscore)
        by_item[item_idx]["hits"].append((pos, INDEX[pos][0], meta["field"], base_score, weight, method, bonus))

    return by_item

def search_items(user_query: str, threshold=70, max_results=8):
    """
    Splits multi-item queries, scores each sub-query, aggregates across sub-queries.
    Returns sorted list of (item, agg_score, debug_rows)
    """
    parts = split_multi_query(user_query.lower())
    if not parts:
        parts = [user_query.lower()]
    parts_norm = [normalize(p) for p in parts]

    agg = defaultdict(lambda: {"score": 0.0, "reasons": []})

    for sub in parts_norm:
        sub_scores = score_one_subquery(sub, score_cutoff=max(40, threshold-20), top_k=80)
        for item_idx, data in sub_scores.items():
            # Aggregate: use softplus to reward multiple sub-query hits without linear explosion
            # softplus(x)=ln(1+e^(x/20))*20, so ~linear around normal score ranges but tempered
            sp = math.log1p(math.exp(data["score"]/20.0)) * 20.0
            agg[item_idx]["score"] += sp
            agg[item_idx]["reasons"].extend(data["hits"])

    # Convert to ranked list
    ranked = []
    for item_idx, data in agg.items():
        # small boost if the canonical item name is in the original raw query
        name = waste_data[item_idx]["item"].lower()
        boost = 6.0 if name in user_query.lower() else 0.0
        final = data["score"] + boost
        ranked.append((item_idx, final, data["reasons"]))

    ranked.sort(key=lambda x: x[1], reverse=True)
    # Filter by a practical threshold (post-aggregation)
    ranked = [r for r in ranked if r[1] >= threshold]
    ranked = ranked[:max_results]

    results = []
    for item_idx, final_score, reasons in ranked:
        results.append((waste_data[item_idx], round(final_score, 1), reasons))
    return results

# ---------- Utilities to render info ----------
def build_maps_links(dropoff_list):
    links = []
    rendered_lines = []
    for loc in dropoff_list:
        name = loc.get("name", "").strip()
        address = (loc.get("address") or "").strip()
        suburb = (loc.get("suburb") or "").strip()
        show_line = f"- {name}" if name else "- Location"
        if address:
            show_line += f", {address}"
        if suburb:
            show_line += f" ({suburb})"
        rendered_lines.append(show_line)

        # Only make GMaps link when we have a mappable address
        if address and ("see website" not in address.lower()) and ("online" not in address.lower()):
            q = urllib.parse.quote_plus(f"{address} {suburb}".strip())
            links.append((name or "Location", f"https://www.google.com/maps/search/?api=1&query={q}"))
    return rendered_lines, links

def render_item_markdown(it, category_notes: dict):
    lines = []
    lines.append(f"### {it['item']}")
    if it.get("category"):
        lines.append(f"**Category:** {it['category']}")
    lines.append(f"**Bin Type:** {it.get('bin_type') or 'Not specified'}")
    lines.append(f"**Recyclable:** {'Yes' if it.get('recyclable') else 'No'}")

    if it.get("instructions"):
        lines.append(f"**What to do:** {it['instructions']}")

    if it.get("drop_off_required") and it.get("drop_off_locations"):
        rendered, links = build_maps_links(it["drop_off_locations"])
        if rendered:
            lines.append("**Drop-off locations:**")
            lines.extend(rendered)
        if links:
            lines.append("\n**📍 Google Maps Links:**")
            for name, url in links:
                lines.append(f"- [{name}]({url})")

    if it.get("extra_notes"):
        lines.append(f"\n**Notes:** {it['extra_notes'].strip()}")

    if it.get("category") in category_notes:
        lines.append("\n**Category Info:**")
        lines.append(category_notes[it["category"]].strip())

    return "\n\n".join(lines)

# ---------- Optional LLM wrapper (uses your prompt style) ----------
from typing import Optional

def explain_with_llm(item_md: str, api_key: Optional[str] = None) -> Optional[str]:
    """
    If api_key provided, use OpenAI Chat Completions to rewrite the markdown into
    a friendly paragraph. Otherwise return None.
    """
    if not api_key:
        return None

    from openai import OpenAI
    client = OpenAI(api_key="")

    sys_prompt = (
        "You are a helpful assistant for the City of Melbourne's waste and recycling guide. "
        "Rewrite the provided info into a friendly, concise explanation with clear steps. "
        "Keep links on their own lines. Avoid prefixes like 'Item:'/'Category:'; just explain."
    )
    user_prompt = "Please explain the following guidance clearly for a resident:\n\n" + item_md

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"system","content":sys_prompt},
                  {"role":"user","content":user_prompt}],
        temperature=0.4,
    )
    return resp.choices[0].message.content.strip()


In [7]:

import gradio as gr

def app_search(user_query, threshold, api_key, show_debug):
    if not user_query or not user_query.strip():
        return "Type something like: `where do I dispose of carpets and batteries?`", pd.DataFrame()

    results = search_items(user_query, threshold=threshold, max_results=8)

    if not results:
        # fallback: show top suggestions even if below threshold
        sugg = search_items(user_query, threshold=max(0, threshold-25), max_results=5)
        if not sugg:
            return "Sorry, I couldn't find recycling information for that item.", pd.DataFrame()
        df = pd.DataFrame([{
            "Suggested Item": r[0]["item"],
            "Category": r[0].get("category",""),
            "Score": r[1],
        } for r in sugg])
        return ("I couldn't find a strong match. Here are some close suggestions you can try:\n" +
                "\n".join(f"- {row['Suggested Item']} ({row['Category']})"
                          for _, row in df.iterrows())), df

    # Build markdown for the best 1–3 matches
    top_to_render = results[:3]
    out_sections = []
    table_rows = []

    for it, sc, reasons in results:
        table_rows.append({
            "Item": it["item"],
            "Category": it.get("category",""),
            "Score": sc
        })

    for it, sc, reasons in top_to_render:
        md = render_item_markdown(it, additional_notes)
        llm_text = explain_with_llm(md, api_key=api_key)
        if llm_text:
            out_sections.append(f"## {it['item']}  \n*(confidence {sc})*\n\n{llm_text}")
        else:
            out_sections.append(f"## {it['item']}  \n*(confidence {sc})*\n\n{md}")

        if show_debug:
            # Show top contributing matches for transparency
            dbg = sorted(reasons, key=lambda r: (r[3]*r[4]), reverse=True)[:6]
            lines = ["\n<details><summary>Why this matched</summary>\n\n| Matched Text | Field | Base Score | Weight | Method | Bonus |",
                     "|---|---:|---:|---:|---|---:|"]
            for (pos, text, field, base, wt, method, bonus) in dbg:
                lines.append(f"| `{text}` | {field} | {base:.1f} | {wt:.2f} | {method} | {bonus:.1f} |")
            lines.append("</details>")
            out_sections.append("\n".join(lines))

    df = pd.DataFrame(table_rows).sort_values("Score", ascending=False)
    return "\n\n---\n\n".join(out_sections), df


with gr.Blocks(title="City of Melbourne – Recycling Helper") as demo:
    gr.Markdown(
        """
        # ♻️ City of Melbourne – Recycling Helper
        Type what you want to dispose of (you can list multiple, e.g. **"carpets, batteries and TV"**).
        """
    )
    with gr.Row():
        query = gr.Textbox(lines=2, label="What do you want to dispose of?", placeholder="e.g., where to dispose of carpets and batteries")
    with gr.Row():
        threshold = gr.Slider(40, 120, value=80, step=1, label="Match confidence (higher = stricter)")
        api_key = gr.Textbox(label="OpenAI API key (optional – for friendlier explanations)", type="password")
        show_debug = gr.Checkbox(label="Show match debug (why it matched)", value=False)

    go = gr.Button("Search")
    out_md = gr.Markdown()
    out_df = gr.Dataframe(interactive=False, label="Matched Items", wrap=True)

    go.click(app_search, inputs=[query, threshold, api_key, show_debug], outputs=[out_md, out_df])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fd45ec7b45353289f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
